In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch
import xarray as xr
import numpy as np
import pandas as pd
import calendar
import seaborn as sns
import seaborn_image as isns

In [2]:
mesh_mask = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/masks/mesh_mask.nc')

In [3]:
mask = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/masks/tmaskm.nc')

In [4]:
mask = mask.tmask.isel(time=0)

### Ocean slicing and mixed layer depth average

In [58]:
# calculate volume for each grid cell

dx = mesh_mask.e1t 
dy = mesh_mask.e2t
grid_cell_area = dx * dy # m2

In [59]:
gridded_mesh = grid_cell_area.isel(t=0)

In [60]:
# apply mask to weighted grid cells

ocean_area = gridded_mesh * mask

In [61]:
ocean_area_sliced = ocean_area.sel(x=slice(520,595),y=slice(330,435),z=0)

In [62]:
ocean_area_sliced_dp = ocean_area_sliced.isel(x=slice(58,60),y=slice(41,43))

In [63]:
### Alkalinity baseline

In [64]:
alk_weighted_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/ALK_baseline_ensemble1_ssp126_1m.nc', decode_times=True)

In [65]:
mld_baseline = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/FOCI2.0-NM004_FM_ESM_SSP126os_2014CC105_1m_grid_T_somxl010_2015to2099.nc')

In [66]:
alk_weighted_baseline = alk_weighted_baseline.sel(time_counter=slice('2090','2100'))

In [9]:
mld_baseline = mld_baseline.sel(time_counter=slice('2090','2100'))

In [10]:
alk_weighted_baseline = alk_weighted_baseline.ALK.isel(x=slice(520,595),y=slice(330,435)) 

In [11]:
mld_baseline = mld_baseline.somxl010.isel(x=slice(520,595),y=slice(330,435)) 

In [12]:
alk_weighted_baseline = alk_weighted_baseline.to_dataset(name='ALK')

In [13]:
mld_baseline = mld_baseline.to_dataset(name='mld')

In [14]:
mld = alk_weighted_baseline.deptht < mld_baseline.mld

In [15]:
mld = mld.to_dataset(name='mld') 

In [16]:
alk_base = alk_weighted_baseline * mld.mld

In [21]:
alk_base = alk_base.where(alk_base)

In [22]:
alk_base = alk_base.fillna(0)

In [23]:
alk_base = alk_base * 1.025 # mmol/m3

In [40]:
# iterate over all layers and append

alk_base_layers = []

for x in list((range(len(alk_base.deptht)))):
    if x == 0:
        alk_n_base = alk_base.ALK.isel(deptht=x) * alk_base.deptht[x] 
        alk_base_layers.append(alk_n_base)
    else:
        alk_n_base = alk_base.ALK.isel(deptht=x) * (alk_base.deptht[x] - alk_base.deptht[x-1])
        alk_base_layers.append(alk_n_base)

In [54]:
weighted_base_alk = sum(alk_base_layers) / mld_baseline.mld # mmol/m3 * m / m

In [55]:
weighted_base_alk = weighted_base_alk.to_dataset(name='ALK')

In [67]:
sliced_alk_baseline = ocean_area_sliced * weighted_base_alk.ALK # baseline

In [68]:
regridded_alk_base = sliced_alk_baseline.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [69]:
regridded_alk_base = regridded_alk_base.to_dataset(name='ALK')

In [32]:
#regridded_alk_base.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/try_base')

### Alkalinity OAE

In [33]:
mld_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/FOCI2.0-TK201_FMESM_SSP126_alkEU_2014CC105_1m_grid_T_somxl010_2015to2099.nc')

In [34]:
alk_weighted_oae = xr.open_dataset('/Volumes/UnionSine/Cropped_DataTransfer/ALK_CDR_EU_coast_ssp126_1m.nc', decode_times=True)

In [35]:
alk_weighted_oae = alk_weighted_oae.sel(time_counter=slice('2090','2100'))

In [36]:
mld_oae = mld_oae.sel(time_counter=slice('2090','2100'))

In [37]:
alk_weighted_oae = alk_weighted_oae.ALK.isel(x=slice(520,595),y=slice(330,435)) 

In [38]:
mld_oae = mld_oae.somxl010.isel(x=slice(520,595),y=slice(330,435)) 

In [39]:
alk_weighted_oae = alk_weighted_oae.to_dataset(name='ALK')

In [40]:
mld_oae = mld_oae.to_dataset(name='mld')

In [41]:
mld = alk_weighted_oae.deptht < mld_oae.mld

In [42]:
mld = mld.to_dataset(name='mld') 

In [43]:
alk_oae = alk_weighted_oae * mld.mld

In [44]:
alk_oae = alk_oae.where(alk_oae)

In [45]:
alk_oae = alk_oae.fillna(0)

In [46]:
alk_oae = alk_oae * 1.025 # mmol/m3

In [47]:
alk_0_oae = alk_oae.ALK.isel(deptht=0) * alk_oae.deptht[0] # mmol/m3 * m
alk_1_oae = alk_oae.ALK.isel(deptht=1) * (alk_oae.deptht[1] - alk_oae.deptht[0])
alk_2_oae = alk_oae.ALK.isel(deptht=2) * (alk_oae.deptht[2] - alk_oae.deptht[1])
alk_3_oae = alk_oae.ALK.isel(deptht=3) * (alk_oae.deptht[3] - alk_oae.deptht[2])
alk_4_oae = alk_oae.ALK.isel(deptht=4) * (alk_oae.deptht[4] - alk_oae.deptht[3])
alk_5_oae = alk_oae.ALK.isel(deptht=5) * (alk_oae.deptht[5] - alk_oae.deptht[4])
alk_6_oae = alk_oae.ALK.isel(deptht=6) * (alk_oae.deptht[6] - alk_oae.deptht[5])
alk_7_oae = alk_oae.ALK.isel(deptht=7) * (alk_oae.deptht[7] - alk_oae.deptht[6])
alk_8_oae = alk_oae.ALK.isel(deptht=8) * (alk_oae.deptht[8] - alk_oae.deptht[7])
alk_9_oae = alk_oae.ALK.isel(deptht=9) * (alk_oae.deptht[9] - alk_oae.deptht[8])

In [48]:
alk_10_oae = alk_oae.ALK.isel(deptht=10) * (alk_oae.deptht[10] - alk_oae.deptht[9])
alk_11_oae = alk_oae.ALK.isel(deptht=11) * (alk_oae.deptht[11] - alk_oae.deptht[10])
alk_12_oae = alk_oae.ALK.isel(deptht=12) * (alk_oae.deptht[12] - alk_oae.deptht[11])
alk_13_oae = alk_oae.ALK.isel(deptht=13) * (alk_oae.deptht[13] - alk_oae.deptht[12])
alk_14_oae = alk_oae.ALK.isel(deptht=14) * (alk_oae.deptht[14] - alk_oae.deptht[13])
alk_15_oae = alk_oae.ALK.isel(deptht=15) * (alk_oae.deptht[15] - alk_oae.deptht[14])
alk_16_oae = alk_oae.ALK.isel(deptht=16) * (alk_oae.deptht[16] - alk_oae.deptht[15])
alk_17_oae = alk_oae.ALK.isel(deptht=17) * (alk_oae.deptht[17] - alk_oae.deptht[16])

In [49]:
alk_18_oae = alk_oae.ALK.isel(deptht=18) * (alk_oae.deptht[18] - alk_oae.deptht[17])
alk_19_oae = alk_oae.ALK.isel(deptht=19) * (alk_oae.deptht[19] - alk_oae.deptht[18])
alk_20_oae = alk_oae.ALK.isel(deptht=20) * (alk_oae.deptht[20] - alk_oae.deptht[19])
alk_21_oae = alk_oae.ALK.isel(deptht=21) * (alk_oae.deptht[21] - alk_oae.deptht[20])
alk_22_oae = alk_oae.ALK.isel(deptht=22) * (alk_oae.deptht[22] - alk_oae.deptht[21])
alk_23_oae = alk_oae.ALK.isel(deptht=23) * (alk_oae.deptht[23] - alk_oae.deptht[22])

In [50]:
weighted_oae_alk = (alk_0_oae + alk_1_oae + alk_2_oae + alk_3_oae + alk_4_oae + alk_5_oae + alk_6_oae +
                     alk_7_oae + alk_8_oae + alk_9_oae + alk_10_oae + alk_11_oae + alk_12_oae + alk_13_oae +
                    alk_14_oae + alk_15_oae + alk_16_oae + alk_17_oae + alk_18_oae + alk_19_oae + alk_20_oae +
                    alk_21_oae + alk_22_oae + alk_23_oae) / mld_oae.mld # mmol/m3 * m / m

In [51]:
weighted_oae_alk = weighted_oae_alk.to_dataset(name='ALK')

In [52]:
sliced_alk_oae = ocean_area_sliced * weighted_oae_alk.ALK # baseline

In [53]:
regridded_alk_oae = sliced_alk_oae.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])

In [54]:
regridded_alk_oae = regridded_alk_oae.to_dataset(name='ALK')

In [ ]:
#regridded_alk_oae.to_netcdf('/Volumes/UnionSine/Cropped_DataTransfer/SSP1_2.6/Alkalinity/try_oae')